<a href="https://githubtocolab.com/giswqs/geemap/blob/master/examples/notebooks/36_quality_mosaic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

Uncomment the following line to install [geemap](https://geemap.org) if needed.

In [2]:
!pip install geemap

  Using cached geemap-0.20.3-py2.py3-none-any.whl (2.1 MB)
  Using cached earthengine_api-0.1.346-py3-none-any.whl
  Using cached pyperclip-1.8.2-py3-none-any.whl
  Using cached geocoder-1.38.1-py2.py3-none-any.whl (98 kB)
  Using cached ee_extra-0.0.15-py3-none-any.whl
  Using cached geeadd-0.5.6-py3-none-any.whl (30 kB)
  Using cached ffmpeg_python-0.2.0-py3-none-any.whl (25 kB)
  Using cached eerepr-0.0.4-py3-none-any.whl (9.7 kB)
  Using cached sankee-0.2.3-py3-none-any.whl
  Using cached google_auth_httplib2-0.1.0-py2.py3-none-any.whl (9.3 kB)
  Using cached google_api_python_client-2.82.0-py2.py3-none-any.whl (11.1 MB)
  Using cached httplib2-0.22.0-py3-none-any.whl (96 kB)
  Using cached logzero-1.7.0-py2.py3-none-any.whl (16 kB)
  Using cached plotly-5.13.1-py2.py3-none-any.whl (15.2 MB)
  Using cached ratelim-0.1.6-py2.py3-none-any.whl (4.0 kB)
  Using cached uritemplate-4.1.1-py2.py3-none-any.whl (10 kB)


# How to find out the greenest day of the year

## Import libraries

In [1]:
import ee
import geemap

In [2]:
# ee.Authenticate()

# Initialize the library.
ee.Initialize()


## Create an interactive map

## Define a region of interest (ROI)

In [3]:
Map = geemap.Map(center=[37.86750755001927, -122.27206838015456], zoom=8)
Map

Map(center=[37.86750755001927, -122.27206838015456], controls=(WidgetControl(options=['position', 'transparent…

Now manually draw an ROI using one of the tools in the map above

In [4]:
roi = ee.FeatureCollection(Map.draw_features)
Map.addLayer(roi, {}, 'roi')
Map.centerObject(roi,8)

## Date of interest

In [180]:
DOI = ee.Date('2022-06-24')
display(DOI.getInfo()['value'])

DOI2 = ee.Date('2021-06-25')
display(DOI2.getInfo()['value'])

#Calculate absolute difference
deltaDays = DOI2.difference(DOI, 'days').abs()
deltaDays.getInfo()

abs(ee.Date(DOI2).getInfo()['value']/1000)

# delta
# diff_2 = DATE_1.difference(DATE_2, 'weeks').getInfo()
# delta = DOI.getInfo()['value'] - DOI.getInfo()['value']

1656028800000

1624579200000

1624579200.0

In [113]:
def addDeltaSeconds(image):
    img_s = image.date().millis() #extract value in ms 
    #Subtract to get difference, take absolute, then negate (because qualityMosaic only does ascending), then convert to seconds
    deltaSeconds = DOI.millis().subtract(img_s).abs().multiply(ee.Number(-1)).divide(ee.Number(1000)).toInt()

    return image.addBands(ee.Image(deltaSeconds).rename('deltaS').toInt())

In [114]:
withDelta = s2Col.map(addDeltaSeconds)

In [86]:
def addDeltaDays(image):
    img_date = ee.Date(image.date())
    deltaDays = ee.Number(DOI.difference(img_date, 'days').abs())
    # img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(deltaDays).rename('delta').toInt())

In [87]:
withDelta = s2Col.map(addDeltaDays)

## Create a quality mosaic

Adapted from [this GEE tutorial](https://developers.google.com/earth-engine/tutorials/community/sentinel-2-s2cloudless)

In [116]:
newest = withDelta.qualityMosaic('deltaS')


Map.addLayer(newest, visParams, 'Newest')

## Same thing, but with a cloud mask

In [10]:
s2Col = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate(start_date, end_date)
    .filterBounds(roi)
    .filterMetadata('CLOUDY_PIXEL_PERCENTAGE', 'less_than', 10)
)
vis_params = {"min": 0, "max": 4000, "bands": ["B5", "B4", "B3"]}

Map.addLayer(s2Col, vis_params, "S2", True)

# QualityMosaic with single param, but masking clouds

Quality moscaic is based on date distance from DOI, and clouds are masked

## Set hyperparams

In [5]:
START_DATE = '2022-01-01'
END_DATE = '2022-12-31'
DOI = ee.Date('2022-06-24')
CLOUD_FILTER = 10
CLD_PRB_THRESH = 20
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50 #m

In [6]:
def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

In [7]:
def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability').rename('prob')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

In [8]:
def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

In [9]:
def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    # return img_cloud_shadow.addBands(is_cld_shdw)
    return img.addBands(is_cld_shdw)

## Apply the cloud and shadow masks

In [184]:
s2_cld_col = get_s2_sr_cld_col(roi, START_DATE, END_DATE)

In [147]:
s2_cld_col_disp = s2_cld_col.map(add_cld_shdw_mask)

In [15]:
s2_cld_col = get_s2_sr_cld_col(roi, START_DATE, END_DATE)

In [16]:
withDelta_cld = s2_cld_col_disp.map(addDeltaSeconds)
newest_cld = withDelta_cld.qualityMosaic('deltaS')

NameError: name 's2_cld_col_disp' is not defined

In [149]:
vis_params = {"min": 0, "max": 4000, "bands": ["B5", "B4", "B3"]}

Map.addLayer(newest_cld.select('cloudmask').selfMask(), {'palette': ['gold']}, 'cloudmask', True)

# QualityMosaic with multiple params

Combine Cloud probability with number of days into a single metric called FScore that is then applied to the qualityMosaic function

In [12]:
s2_cld_col = get_s2_sr_cld_col(roi, START_DATE, END_DATE)
visParams = {"min": 0, "max": 4000, "bands": ["B5", "B4", "B3"]}
Map.addLayer(s2_cld_col, visParams, 'col')

In [13]:
K = 1000
OFFSET = -0.5
MULT = 100

In [19]:
def addDate(image):
    # img_date = ee.Date(image.date())
    # img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    # img_s =  #extract value in ms 
    #Subtract to get difference, take absolute, then negate (because qualityMosaic only does ascending), then convert to days
    deltaDays = DOI.millis().subtract(image.date().millis()).abs().divide(ee.Number(8.64e+7)).toInt()
    return image.addBands(ee.Image(deltaDays).rename('date').toInt())

In [14]:
def addFScore(image):
    # img_cloud = add_cloud_bands(image) #now it had a band called 'prob'
    image = add_cloud_bands(image)
    
    #Subtract to get difference, take absolute, then negate (because qualityMosaic only does ascending), then convert to days
    deltaDays = DOI.millis().subtract(image.date().millis()).abs().divide(ee.Number(8.64e+7)).toInt()
    image = image.addBands(ee.Image(deltaDays).rename('delta').toInt())
    
    # image.addBands(ee.Image(deltaDays)).rename('deltaDays')
    # image = image.addBands(ee.Image(deltaDays).rename('delta'))
    prob = image.select('prob').divide(100)
    # image.addBands(prob).rename('prob')
    
    x = ee.Image(deltaDays.add(1).log10()).add(1).multiply(ee.Image(K).pow(prob.add(ee.Number(OFFSET)))).multiply(ee.Image(MULT))
    # x = ee.Image(deltaDays.log10()).toInt64().multiply(-1)
    return image.addBands(x.multiply(ee.Number(-1)).rename('FScore'))#.addBands(prob)

#     #Calculate F score
#     fScore = deltaDays.log10().add(1).multiply(ee.Number(K).pow(prob.add(ee.Number(OFFSET)))).multiply(MULT).toInt()
    
    
#     return image.addBands(ee.Image(fScore).rename('FScore'))


In [15]:
witFScore = s2_cld_col.map(addFScore)

newest_cldprob = witFScore.qualityMosaic('FScore')
# newest_cldprob = witFScore.qualityMosaic('prob')

In [16]:
newest_cldprob.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60',
 'prob',
 'clouds',
 'delta',
 'FScore']

In [17]:
visParams = {
    'bands': ['B4', 'B3', 'B2'],
    'min': 0,
    'max': 4000,
}
Map.addLayer(newest_cldprob, visParams, 'FScore')

In [18]:
withDate = s2_cld_col.map(addDate).mosaic()
Map.addLayer(withDate, visParams, 'Date')

NameError: name 'addDate' is not defined

In [93]:
withDate.bandNames().getInfo()

['B1',
 'B2',
 'B3',
 'B4',
 'B5',
 'B6',
 'B7',
 'B8',
 'B8A',
 'B9',
 'B11',
 'B12',
 'AOT',
 'WVP',
 'SCL',
 'TCI_R',
 'TCI_G',
 'TCI_B',
 'MSK_CLDPRB',
 'MSK_SNWPRB',
 'QA10',
 'QA20',
 'QA60',
 'date']